# Team 3 - Merchant Category Recommendation
## Modeling

### Team 3
- Vinicio De Sola
- Kevin Hanna
- Pri Nonis
- Bradley Nott


The results from Feature Extraction should influence the modeling done in this notebook. 

In [1]:
import numpy               as np
import matplotlib.pyplot   as plt
import pandas              as pd
import seaborn             as sb

from datetime              import timedelta, datetime

from sklearn               import metrics
from sklearn.decomposition import PCA
from sklearn.cluster       import KMeans
from sklearn.mixture       import GaussianMixture
from sklearn.linear_model  import LinearRegression

from matplotlib.colors     import LogNorm

from IPython.display       import HTML, Markdown

from sklearn.metrics       import mean_squared_error

%matplotlib inline

np.random.seed(0)

import os
print(os.listdir("./input"))

['historical_transactions.csv', '.ipynb_checkpoints', 'new_merchant_transactions.csv', 'engineered_test.pkl', '.~lock.Data_Dictionary.xlsx#', 'merchants.csv', 'sample_submission.csv', 'test.csv', 'Data_Dictionary.xlsx', 'engineered_train.pkl', '.gitignore', 'train.csv']


## Read in all the data files

In [2]:
train        = pd.read_pickle('input/engineered_train.pkl')
test         = pd.read_pickle('input/engineered_test.pkl')

target = train['target']
del train['target']

## A view of our training data dataframe

In [3]:
display(train.head())
display(test.head())
display(target.head())

,feature_3,first_active_delta_days,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,...,new_category_2_5.0_sum,new_category_3_A_sum,new_category_3_B_sum,new_category_3_C_sum,new_authorized_flag_y_sum,new_authorized_flag_n_sum,new_month_diff_mean,new_month_diff_min,new_month_diff_max,new_month_diff_std
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,1,246,0,0,0,0,1,0,1,0,...,0.0,23.0,0.0,0.0,23.0,0.0,13.521739,13.0,14.0,0.510754
C_ID_3d0044924f,0,397,0,0,0,1,0,1,0,0,...,0.0,0.0,6.0,0.0,6.0,0.0,14.500000,14.0,15.0,0.547723
C_ID_d639edf6cd,0,550,0,1,0,0,0,0,1,0,...,1.0,1.0,0.0,0.0,1.0,0.0,13.000000,13.0,13.0,0.000000
C_ID_186d6a6901,0,154,0,0,0,1,0,0,0,1,...,0.0,0.0,6.0,0.0,7.0,0.0,13.571429,13.0,14.0,0.534522
C_ID_cdbd2c0db2,0,93,1,0,0,0,0,0,0,1,...,0.0,0.0,34.0,1.0,36.0,0.0,13.611111,13.0,14.0,0.494413


,first_active_month,feature_3,first_active_delta_days,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,...,new_category_2_5.0_sum,new_category_3_A_sum,new_category_3_B_sum,new_category_3_C_sum,new_authorized_flag_y_sum,new_authorized_flag_n_sum,new_month_diff_mean,new_month_diff_min,new_month_diff_max,new_month_diff_std
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_0ab67a22ab,2017-04-01,1,307,0,0,1,0,0,0,0,...,0.0,0.0,2.0,1.0,3.0,0.0,15.333333,15.0,16.0,0.577350
C_ID_130fd0cbdd,2017-01-01,0,397,0,1,0,0,0,0,0,...,0.0,0.0,8.0,1.0,9.0,0.0,13.777778,13.0,14.0,0.440959
C_ID_b709037bc5,2017-08-01,1,185,0,0,0,0,1,1,0,...,1.0,0.0,1.0,1.0,2.0,0.0,14.000000,14.0,14.0,0.000000
C_ID_d27d835a9f,2017-12-01,0,63,0,1,0,0,0,1,0,...,1.0,0.0,5.0,5.0,10.0,0.0,13.600000,13.0,14.0,0.516398
C_ID_2b5e3df5c2,2015-12-01,1,794,0,0,0,0,1,1,0,...,0.0,0.0,4.0,1.0,6.0,0.0,14.000000,14.0,14.0,0.000000


card_id
C_ID_92a2005557   -0.820283
C_ID_3d0044924f    0.392913
C_ID_d639edf6cd    0.688056
C_ID_186d6a6901    0.142495
C_ID_cdbd2c0db2   -0.159749
Name: target, dtype: float64

## Train and test model using ALL engineered features 

This is where the results from Elo_feature_Extraction should be as a starting point.

In [4]:
def calculate_engineered():
    np.random.seed(0)
#     training = train[train.columns[1:]]
#     del training['target']
#     labels = train[['target']]
    
    data_len = len(train)
    mask = np.random.rand(data_len) < 0.8

    train_data = train.values[mask]
    train_labels = target.values.flatten()[mask]
    test_data = train.values[~mask]
    test_labels = target.values.flatten()[~mask]

    X = train_data
    y = train_labels

    X_test = test_data
    y_test = test_labels

    reg = LinearRegression()
    reg.fit(X, y)

    y_pred_reg = reg.predict(X_test)
    mse = mean_squared_error(y_test, y_pred_reg)

    y_mean = train_labels.mean()
    y_test_mean = test_labels.mean()

    print("After doing feature enginnering, the Mean Squared Error is: {:0.3f}".format(mse))
    
    return mse


engineered_mse = calculate_engineered()

After doing feature enginnering, the Mean Squared Error is: 13.902


In [5]:
train.columns

Index(['feature_3', 'first_active_delta_days', 'feature_1_1', 'feature_1_2',
       'feature_1_3', 'feature_1_4', 'feature_1_5', 'feature_2_1',
       'feature_2_2', 'feature_2_3', 'old_purchase_amount_sum',
       'old_purchase_amount_mean', 'old_purchase_amount_median',
       'old_subsector_id_nunique', 'old_merchant_category_id_nunique',
       'old_merchant_id_nunique', 'old_city_id_nunique',
       'old_state_id_nunique', 'old_category_1_sum', 'old_category_2_1.0_sum',
       'old_category_2_2.0_sum', 'old_category_2_3.0_sum',
       'old_category_2_4.0_sum', 'old_category_2_5.0_sum',
       'old_category_3_A_sum', 'old_category_3_B_sum', 'old_category_3_C_sum',
       'old_authorized_flag_y_sum', 'old_authorized_flag_n_sum',
       'old_month_diff_mean', 'old_month_diff_min', 'old_month_diff_max',
       'old_month_diff_std', 'new_purchase_amount_sum',
       'new_purchase_amount_mean', 'new_purchase_amount_median',
       'new_subsector_id_nunique', 'new_merchant_category_id_nu

## Test area to play with different sets of features

In [6]:
def feature_play():
    np.random.seed(0)
    #This is just to play around with different engineered features. 

    
    tmp_train = train[[
        # Train
        'feature_3',
        'first_active_delta_days',
        'feature_1_1', 
        'feature_1_2', 
        'feature_1_3', 
        'feature_1_4',
        'feature_1_5', 
        'feature_2_1', 
        'feature_2_2', 
        'feature_2_3',
        
        # Old
        'old_purchase_amount_sum', 
        'old_purchase_amount_mean',
#         'old_purchase_amount_median', # Mean is better than Median
        'old_subsector_id_nunique',
        'old_merchant_category_id_nunique', 
        'old_merchant_id_nunique',
        'old_city_id_nunique', 
        'old_state_id_nunique', 
        'old_category_1_sum',
        'old_category_2_1.0_sum', 
        'old_category_2_2.0_sum',
        'old_category_2_3.0_sum', 
        'old_category_2_4.0_sum',
        'old_category_2_5.0_sum', 
        'old_category_3_A_sum',
        'old_category_3_B_sum', 
        'old_category_3_C_sum',
        'old_authorized_flag_y_sum',
        'old_authorized_flag_n_sum',
        'old_month_diff_mean', 
        'old_month_diff_min', 
        'old_month_diff_max',
        'old_month_diff_std',
        
        # New
        'new_purchase_amount_sum', 
        'new_purchase_amount_mean',
#         'new_purchase_amount_median', 
        'new_subsector_id_nunique',
        'new_merchant_category_id_nunique', 
        'new_merchant_id_nunique',
        'new_city_id_nunique', 
        'new_state_id_nunique', 
        'new_category_1_sum',
        'new_category_2_1.0_sum', 
        'new_category_2_2.0_sum',
        'new_category_2_3.0_sum', 
        'new_category_2_4.0_sum',
        'new_category_2_5.0_sum', 
        'new_category_3_A_sum',
        'new_category_3_B_sum', 
        'new_category_3_C_sum',
        'new_authorized_flag_y_sum',
        'new_authorized_flag_n_sum',
        'new_month_diff_mean', 
        'new_month_diff_min', 
        'new_month_diff_max',
        'new_month_diff_std'
    ]]
    
    data_len = len(tmp_train)
    mask = np.random.rand(data_len) < 0.8

    train_data = tmp_train.values[mask]
    train_labels = target.values.flatten()[mask]
    test_data = tmp_train.values[~mask]
    test_labels = target.values.flatten()[~mask]

    X = train_data
    y = train_labels

    X_test = test_data
    y_test = test_labels

    reg = LinearRegression()
    reg.fit(X, y)

    y_pred_reg = reg.predict(X_test)
    mse = mean_squared_error(y_test, y_pred_reg)

    y_mean = train_labels.mean()
    y_test_mean = test_labels.mean()

    print("Using the following features, the Mean Squared Error is: {:0.3f}".format(mse))
    print(tmp_train.columns)
    
    return mse

# if play_mse:
#     print("Previous MSE: {:0.3f}".format(play_mse))
    
play_mse = feature_play()

Using the following features, the Mean Squared Error is: 13.884
Index(['feature_3', 'first_active_delta_days', 'feature_1_1', 'feature_1_2',
       'feature_1_3', 'feature_1_4', 'feature_1_5', 'feature_2_1',
       'feature_2_2', 'feature_2_3', 'old_purchase_amount_sum',
       'old_purchase_amount_mean', 'old_subsector_id_nunique',
       'old_merchant_category_id_nunique', 'old_merchant_id_nunique',
       'old_city_id_nunique', 'old_state_id_nunique', 'old_category_1_sum',
       'old_category_2_1.0_sum', 'old_category_2_2.0_sum',
       'old_category_2_3.0_sum', 'old_category_2_4.0_sum',
       'old_category_2_5.0_sum', 'old_category_3_A_sum',
       'old_category_3_B_sum', 'old_category_3_C_sum',
       'old_authorized_flag_y_sum', 'old_authorized_flag_n_sum',
       'old_month_diff_mean', 'old_month_diff_min', 'old_month_diff_max',
       'old_month_diff_std', 'new_purchase_amount_sum',
       'new_purchase_amount_mean', 'new_subsector_id_nunique',
       'new_merchant_category_i